# Analyzing Parade

As a board game enjoyer, I recently discovered an excellent card game called [Parade](https://boardgamegeek.com/boardgame/56692/parade). It is not that different from [Arboretum](https://boardgamegeek.com/boardgame/140934/arboretum), except it's more random so less cutthroat. Basically, you try to take as few cards as possible from the parade because cards score negative points. After playing a couple games, I came up with some hypothesis about the game's strategy and mechanics.

Unlike Arboretum, where the general strategy is to play small discard small. It is hard to come up with a strategy for Parade. It feels very random. I decided to simulate the game and see what I could come up with. I impelemented the simulator in Rust to gather some statistics using a 90%-there strategy. It's not perfect, but it's good enough to give some ideas about Parade's characteristics, especially just how much does the early game affects the game's outcome. I also wanted to optimize to avoid being forced to take cards from the parade, which is a bad experience. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.ticker import PercentFormatter
import itertools

The simulator takes different parameters. players count, number of suits, number of ranks, number of iterations, and strategies. The strategy is a function that takes a player's hand and returns a card to add to the parade. The simulator runs the game for a number of iterations and outputs the score of the game. Then in pandas, I calculate the statistics for configuration.

In [2]:
def run(players=2, suits=6, ranks=11, iters=50000, strats=[1]):
    result = []
    for s in itertools.product(strats, repeat=players):
        strat = " ".join(list(map(str,s)))
        ! cargo run --release -- --players=$players --suits=6 --ranks=11 --iters=$iters $strat
        df = pd.read_csv("output.csv", header=None)
        df = df.set_axis(["score "+str(i) for i in range(players)]+ ["forced take "+str(i) for i in range(players)], axis=1)
        means = df.mean()
        for i, m in enumerate(s): 
            means[str(i)] = m
        result.append(means)
    return pd.DataFrame(result)

I wrote 3 strategies:

1. Always use the first card in hand.
2. For each card in hand, find the lowest sum of ranks from the ejected cards.
3. On top of 2, voluntarily take rank 0-2 cards from the parade.

In [5]:
df = run(players=2, suits=6, ranks=11, iters=5000, strats=[0, 1, 2])
df['scorediff'] = df['score 0'] - df['score 1']
df['forcedtakediff'] = df['forced take 0'] - df['forced take 1']
df

    Finished release [optimized] target(s) in 0.03s
     Running `target/release/parade --players=2 --suits=6 --ranks=11 --iters=5000 0 0`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=6 --ranks=11 --iters=5000 0 1`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=6 --ranks=11 --iters=5000 0 2`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=6 --ranks=11 --iters=5000 1 0`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=6 --ranks=11 --iters=5000 1 1`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=6 --ranks=11 --iters=5000 1 2`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=6 --ranks=11 --iters=5000 2 0`
    Finished release [optim

,score 0,score 1,forced take 0,forced take 1,0,1,scorediff,forcedtakediff
0,80.8092,95.4862,3.3016,3.4592,0.0,0.0,-14.6770,-0.1576
1,146.9090,35.2912,6.9034,12.0610,0.0,1.0,111.6178,-5.1576
2,145.3782,36.1774,6.8086,11.7340,0.0,2.0,109.2008,-4.9254
3,26.9696,161.1628,11.7688,7.1186,1.0,0.0,-134.1932,4.6502
4,71.0302,88.1068,16.3072,16.6150,1.0,1.0,-17.0766,-0.3078
5,70.0976,88.9962,16.2268,16.4954,1.0,2.0,-18.8986,-0.2686
6,27.6018,160.3182,11.4250,7.0140,2.0,0.0,-132.7164,4.4110
7,71.4762,87.6924,16.1492,16.5404,2.0,1.0,-16.2162,-0.3912
8,70.7912,88.4290,16.1066,16.4522,2.0,2.0,-17.6378,-0.3456


In [8]:
df.pivot_table(index="0", columns="1", values='scorediff', fill_value=0)

1,0.0,1.0,2.0
0,,,
0.0,-14.6770,111.6178,109.2008
1.0,-134.1932,-17.0766,-18.8986
2.0,-132.7164,-16.2162,-17.6378


In [9]:
df.pivot_table(index="0", columns="1", values='forcedtakediff', fill_value=0)

1,0.0,1.0,2.0
0,,,
0.0,-0.1576,-5.1576,-4.9254
1.0,4.6502,-0.3078,-0.2686
2.0,4.4110,-0.3912,-0.3456


In [6]:
df = run(players=3, suits=6, ranks=11, iters=5000)
df['score std'] = df.loc[:, df.columns.str.startswith('score')].loc[0].mean()
df['forced take std'] = df.loc[:, df.columns.str.startswith('forced take')].loc[0].mean()
df

    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=3 --suits=6 --ranks=11 --iters=5000 1 1 1`


,score 0,score 1,score 2,forced take 0,forced take 1,forced take 2,0,1,2,score std,forced take std
0,50.2918,56.824,61.1084,9.4528,9.6584,9.8874,1.0,1.0,1.0,56.074733,9.6662
